# CONTROL DE HABERES (partida 100)

La finalidad del presente reporte es cruzar la información de ejecución de obras entre Icaro y el Resumen de Rendiciones por Proveedor del SGF. En la medida que la información de Icaro haya sido previamente cruzada con SIIF, este control permitirá detectar, en mayor medida, desvíos en la carga del SIIF.

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [ ]:
from invicoctrlpy.gastos.control_haberes.control_haberes import ControlHaberes

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

from  pivottablejs import pivot_ui

EJERCICIO = '2022'
UPDATE_DB = False
control_haberes = ControlHaberes(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Controles

### Control por Mes

El presente reporte agrupa la información del SIIF y el SSCC por mes para luego cruzarlos. En las últimas columnas se exponen las diferencias detectas entre ambos sistemas en relación al pago de haberes de forma mensual y acumalada. 

In [ ]:
control_mes = control_haberes.control_mes()
itables.show(control_mes, column_filters="footer", dom="lrtip")

## Tablas Auxiliares

### Comprobantes SIIF de Haberes (rcg01_uejp + gto_rpa03g)

Los comprobantes gastos del SIIF listado a continuación únicamente corresponden a los registrados con la cuenta 130832-04 (Haberes). Además, están neteados de ART (partidas 150 y 151) porque, en la realidad, no se pagan con la 130832-04 y esto dificulta el cruzamiento de datos. Por el mismo motivo también se netea los registros relacionados a la liquidación de retención gcias 4ta categoría (cod. ret. 245) y los ajuestes por haberes erróneos (cod. ret. 310)

In [ ]:
siif = control_haberes.siif_comprobantes_haberes_neto_rdeu
itables.show(siif, column_filters="footer", dom="lrtip")

### Deuda Flotante SIIF (rdeu012)

La presente tabla expone la deuda flotante del ejercicio relacionada a la liquidación de haberes

In [ ]:
rdeu = control_haberes.siif_rdeu012
itables.show(rdeu, column_filters="footer", dom="lrtip")

### Resumen General de Movimentos (Sist. Seg. Ctas. Ctes. INVICO)

El listado que a continuación se detalla corresponde a los movimientos del cuenta 130832-04, excluidos los depósitos, del Sistema de Seguimiento de Cuentas Corrientes (SSCC o Banco INVICO). Además, los movimientos están neteados de los correspondientes al concepto *ganancias* dado que, en ocasiones, los mismos se abonan con otra cuenta distinta a la 130832-04.

In [ ]:
sscc = control_haberes.sscc_banco_invico
itables.show(sscc, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [ ]:
with pd.ExcelWriter('control_haberes.xlsx') as writer:
    control_mes.to_excel(writer, sheet_name='siif_vs_sscc', index=False)
    siif.to_excel(writer, sheet_name='siif', index=False)
    sscc.to_excel(writer, sheet_name='sscc', index=False)
    rdeu.to_excel(writer, sheet_name='rdeu', index=False)

### Tablas Dinámicas

Por separado, se incluye una tabla dinámica con toda la información de SIIF y del SSCC para el ejercicio bajo análisis. Ambas bases de datos fueron pivoteadas en función del campo mes. A su vez, se incluyó un campo adicional, sobre el cual debería realizarse los cálculos agregados, llamado **diferencia** el cual surge de resta entre la ejecución SIIF y el importe pagado del SSCC.

In [ ]:
pivot_ui(control_haberes.control_completo(), 'tabla_dinamica_control_haberes.html')